In [12]:
%pip install -r requirements.txt
# or 
# !pip install -U .

Note: you may need to restart the kernel to use updated packages.


# Example Run of an Augmented Simulator

This tutorial notebook provides a guidance for installing the required packages and testing implemented augmented simulators using LIPS platform. 

**A quick walkthrough:**

- Install the required packages using the requirements.txt file in the github repository for the required used case.

- Some baseline are already implemented in the LIPS platform that could be seen to have some inspiration.

- The augmented simulators related hyperparameters could be modified via dedicated configuration files.

- The LIPS platform will be used to evaluate the trained augmented simulators from different evaluation criteria categories and attribute a score to each run.

## How to implement your own augmented simulator

In the following, we show 3 ways to implement an augmented simulator (based on ML or a hybrid physics-AI model):

1- Using an existing augmented simulator (baseline) in LIPS platform, train it and then evaluate the results;

2- Implement an augmented simulator using LIPS framework template to take the advantage of existing training loop and other offered features;

3- Implement an augmented simulator independently from LIPS platform and plug the trained model into LIPS to evaluate its results.

As so, in order to adress the augmented simulator handling when running this notebook, it is divided into the following four sections:
1. [Generic step (Load the required data)](#generic_step)
2. [Evaluate an existing augmented simulator](#existing_sim) (Beginner users)
3. [Train and evaluate a custom augmented simulators developed using LIPS framework](#train_using_lips) (Intermediate level users)
4. [Train a custom augmented simulator independently from LIPS and use the framwork to evaluate the final results](#train_custom) (Advanced users)

Depending on the user level, it conveniently point to the dedicated section.

### Prerequisites

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS) 

Install the AirfRANS package

In [13]:
# !pip install airfrans

### Generic Step (Load the required data) <a id='generic_step'></a>

In [1]:
import os
from lips import get_root_path

In [2]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [3]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations","simulators","torch_fc.ini") #Configuration file re

Download the data

In [4]:
from lips.dataset.airfransDataSet import download_data
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

/home/dimitri/Development/AI/cfdml/env/lib/python3.8/site-packages/grid2op/Backend/PandaPowerBackend.py:31: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	/home/dimitri/Development/AI/cfdml/env/bin/python -m pip install numba

  warnings.warn(


Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [5]:
# Load the required benchmark datasets
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path=LOG_PATH)
                            
benchmark.load(path=DIRECTORY_NAME)

Loading dataset (task: reynolds, split: test): 100%|██████████| 496/496 [01:58<00:00,  4.17it/s]


In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [9]:
def process_dataset(dataset, batch_size: int=128000, training: bool=False, shuffle: bool=False, pin_memory=True, num_workers=0, dtype=torch.float32) -> DataLoader:
    if training:
        batch_size = batch_size
        extract_x, extract_y = dataset.extract_data()
    else:
        batch_size = batch_size
        extract_x, extract_y = dataset.extract_data()

    torch_dataset = TensorDataset(torch.tensor(extract_x, dtype=dtype), torch.tensor(extract_y, dtype=dtype))
    data_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory, num_workers=num_workers)
    return data_loader

def infer_input_output_size(dataset):
    *dim_inputs, output_size = dataset.get_sizes()
    input_size = np.sum(dim_inputs)
    return input_size, output_size

train_loader = process_dataset(benchmark.train_dataset, training=True)